<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TP2_automatico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carga de librerias

In [ ]:
!pip install ebooklib beautifulsoup4 pandas

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from typing import List, Dict, Any
import numpy as np
import pandas as pd
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import re

### Creacion de datos

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def normalize_text_y(t):
    # Convertir a minúsculas y quitar puntuación
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-Záéíóúüñ0-9¿?,.' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [ ]:
def transformar_etiqueta(y, indice):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(indice)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append(1) #('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append(0) #("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append(3) #('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append(1) #('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append(2) #('","')
          else:
            puntuacion_finales.append(0) #("")

        else:
          puntuacion_finales.append(0) #("")

        if palabra.islower():
          capitalizaciones.append(0)
          ultimo_numero = 0
        elif palabra.istitle():
          capitalizaciones.append(1)
          ultimo_numero = 1
        elif palabra.isupper():
          capitalizaciones.append(3)
          ultimo_numero = 3
        else:
          capitalizaciones.append(2)
          ultimo_numero = 2

  etiquetas = np.column_stack([instancia_ids, token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [ ]:
def convertir_epub_a_csv(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

In [ ]:
def crearDataSet(libro):
  convertir_epub_a_csv(libro)
  df = pd.read_csv('libro_parrafos.csv')
  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  columnas = ['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización']
  datos = pd.DataFrame(columns=columnas)

  i = 0
  for p in parrafos['default']:
    etiquetas = transformar_etiqueta([p], i)
    etiquetas = pd.DataFrame(etiquetas, columns=columnas)
    datos = pd.concat([datos, etiquetas], ignore_index=True)
    i += 1

  print('Data set creado de tamaño: ', datos.shape)

  numeric_cols = ['instancia_id', 'token_id', 'punt_inicial', 'punt_final', 'capitalización']
  for col in numeric_cols:
    datos[col] = pd.to_numeric(datos[col], errors='coerce')

  return parrafos, datos

In [ ]:
def agregar_embeddings(df_X):
  token_ids = df_X['token_id'].tolist()
  embeddings_list = []
  for token_id  in token_ids:
      if token_id is None or token_id == tokenizer.unk_token_id:
        token_id = tokenizer.unk_token_id
      # Detach the tensor before converting to list
      embedding = model.embeddings.word_embeddings.weight[token_id].detach().tolist()
      embeddings_list.append(embedding)
  df_X['embeddings'] = embeddings_list
  # The embeddings are already lists of floats, no need to convert to int
  # df_X['embeddings'] = df_X['embeddings'].apply(lambda x: [int(i) for i in x])

  return df_X

In [ ]:
def trasformar_df_dfPyTorch(datos_X, datos_Y):
  # Convert the list of lists in the 'embeddings' column to a NumPy array of floats
  embeddings_array = np.array(datos_X['embeddings'].tolist(), dtype=np.float32)
  X = torch.tensor(embeddings_array, dtype=torch.float32)
  Y = torch.tensor(datos_Y[['punt_inicial', 'punt_final', 'capitalización']].values, dtype=torch.float32)
  dataSetPT = TensorDataset(X, Y)

  return dataSetPT

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'


In [ ]:
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'

In [ ]:
path = path + libro1


In [ ]:
!wget -O libro1.epub $path


In [ ]:
parrafos, dataSet = crearDataSet('libro1.epub')

In [ ]:
parrafos.head()

In [ ]:
parrafos.iloc[100]

In [ ]:
dataSet.loc[dataSet["instancia_id"] == 100]

In [ ]:
datos_X = dataSet[['instancia_id', 'token_id', 'token']]
datos_Y = dataSet[['punt_inicial', 'punt_final', 'capitalización']]

In [ ]:
datos_X_ext = agregar_embeddings(datos_X)

In [ ]:
dataSetPT = trasformar_df_dfPyTorch(datos_X, datos_Y)


### Padding

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    batch: lista de tuplas (embeddings, labels)
    """
    embeddings_list, labels_list = zip(*batch)

    # Pad embeddings (seq_len, embedding_dim) -> (batch_size, max_seq_len, embedding_dim)
    embeddings_padded = pad_sequence(embeddings_list, batch_first=True, padding_value=0.0)

    # Pad labels
    punt_inicial = pad_sequence([l["punt_inicial"] for l in labels_list], batch_first=True, padding_value=-100)
    punt_final = pad_sequence([l["punt_final"] for l in labels_list], batch_first=True, padding_value=-100)
    capitalizacion = pad_sequence([l["capitalización"] for l in labels_list], batch_first=True, padding_value=-100)

    return embeddings_padded, {
        "punt_inicial": punt_inicial,
        "punt_final": punt_final,
        "capitalizacion": capitalizacion
    }

### Data Loader

In [ ]:
dataloader = DataLoader(dataSetPT, batch_size=8, shuffle=True, collate_fn=collate_fn)


## Modelos

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datos_X, datos_Y, test_size=0.2, random_state=42)

In [ ]:
X_train['embeddings']

In [ ]:
X_train["embeddings_mean"] = X_train["embeddings"].apply(lambda x: np.mean(x))


In [ ]:
X_test["embeddings_mean"] = X_test["embeddings"].apply(lambda x: np.mean(x))


In [ ]:
    model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

    #n_estimators cantidad de arboles
    # max_depth altura maxima de cada uno


In [ ]:
y_test

In [ ]:
model.fit(X_train[['embeddings_mean']], y_train)


In [ ]:
y_pred = model.predict(X_test[['embeddings_mean']])
y_pred

In [ ]:
f1_score(y_test['punt_inicial'], y_pred[:,0],average="macro")

In [ ]:
f1_score(y_test['punt_inicial'],y_pred[:,1],average = "macro")

In [ ]:
f1_score(y_test['capitalización'],y_pred[:,2],average = "macro")

### RNN Unidireccional

In [ ]:
class EncoderUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,#como es 2, significa que hay dos bloques de celdas LSTM
            batch_first=True, #(batch, seq, feature)
            dropout=dropout, #dropout probability
            bidirectional=False  # unidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)


In [ ]:
class DecoderUnidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderUnidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False  # unidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)


        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }

#### Encoder - Decoder

In [ ]:
class ModeloUnidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloUnidireccional, self).__init__()
        self.encoder = EncoderUnidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderUnidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions

### RNN Bidireccional

In [ ]:
class EncoderBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(EncoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

    def forward(self, embeddings):
        """
        embeddings: tensor de forma (batch_size, seq_len, embedding_dim)
        """
        outputs, (hidden, cell) = self.lstm(embeddings)
        # outputs: (batch_size, seq_len, hidden_dim)
        # hidden: (num_layers, batch_size, hidden_dim)
        # cell:   (num_layers, batch_size, hidden_dim)
        return outputs, (hidden, cell)

In [ ]:
class DecoderBidireccional(nn.Module):
    def __init__(self, hidden_dim=256, num_layers=2, dropout=0.3):
        super(DecoderBidireccional, self).__init__()
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True  # bidireccional
        )

        # Capa feed-forward para cada problema
        self.punt_inicial_ff = nn.Linear(hidden_dim, 2)
        self.punt_final_ff = nn.Linear(hidden_dim, 4)
        self.capital_ff = nn.Linear(hidden_dim, 4)

        # Función de activación para cada problema
        #self.punt_inicial_sigmoid = nn.Sigmoid()
        #self.punt_final_softmax = nn.Softmax(dim=4)
        #self.capital_softmax = nn.Softmax(dim=4)


    def forward(self, encoder_outputs, hidden, cell):
        """
        encoder_outputs: (batch_size, seq_len, hidden_dim)
        hidden, cell: del encoder
        """
        outputs, _ = self.lstm(encoder_outputs, (hidden, cell))

        #punt_inicial_logits = self.punt_inicial_sigmoid(self.punt_inicial_ff(outputs))
        #punt_final_logits = self.punt_final_sofmax(self.punt_final_ff(outputs))
        #capital_logits = self.capital_sofmax(self.capital_ff(outputs))

        punt_inicial_logits = self.punt_inicial_ff(outputs)
        punt_final_logits = self.punt_final_ff(outputs)
        capital_logits = self.capital_ff(outputs)

        return {
            "puntuación inicial": punt_inicial_logits,
            "puntuación final": punt_final_logits,
            "capitalización": capital_logits,
        }


#### Encoder - Decoder bidireccional

In [ ]:
class ModeloBidireccional(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=256, num_layers=2, dropout=0.3):
        super(ModeloBidireccional, self).__init__()
        self.encoder = EncoderBidireccional(embedding_dim, hidden_dim, num_layers, dropout)
        self.decoder = DecoderBidireccional(hidden_dim, num_layers, dropout)

    def forward(self, embeddings):
        encoder_outputs, (hidden, cell) = self.encoder(embeddings)
        predictions = self.decoder(encoder_outputs, hidden, cell)
        return predictions

### Entrenamiento

In [ ]:
model = ModeloUnidireccional(embedding_dim=768, hidden_dim=256, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)  # ignorar padding
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for embeddings, labels in dataloader:
        optimizer.zero_grad()

        outputs = model(embeddings)  # diccionario con tus tres salidas

        loss_inicial = criterion(outputs["puntuación inicial"].permute(0,2,1), labels["punt_inicial"])
        loss_final = criterion(outputs["puntuación final"].permute(0,2,1), labels["punt_final"])
        loss_cap = criterion(outputs["capitalización"].permute(0,2,1), labels["capitalizacion"])

        loss = loss_inicial + loss_final + loss_cap
        loss.backward()
        optimizer.step()